In [2]:
import pandas as pd 

In [5]:
from serpapi import GoogleSearch

params = {
  "q": "hurricane harvey fema",
  "tbm": "nws",
  "num": "200",
  "api_key": "a97eba3a033f03499351a58a0480ff24163ff9ed5b37783b872e81f2213e0e45"
}

search = GoogleSearch(params)
results = search.get_dict()
news_results = results['news_results']

In [6]:
links = []
dates = []
titles = []
sources = []
for result in news_results:
    link = result['link']
    date = result['date']
    title = result['title']
    source = result['source']
    links.append(link)
    dates.append(date)
    titles.append(title)
    sources.append(source)

In [7]:
harvey_df = pd.DataFrame({
    "Title": titles,
    "Source": sources,
    "Date": dates,
    "Link": links
})
harvey_df.head()

,Title,Source,Date,Link
0,FEMA overhauls the National Flood Insurance Pr...,CNBC,2 days ago,https://www.cnbc.com/2021/08/17/fema-overhauls...
1,Fix Disaster Response Now,Scientific American,2 days ago,https://www.scientificamerican.com/article/fix...
2,"Meet Marcus Coleman, New Director of DHS Cente...",| FEMA.gov,2 weeks ago,https://www.fema.gov/blog/meet-marcus-coleman-...
3,Why Biden might be spending too much on climate,E&E News,1 day ago,https://www.eenews.net/articles/why-biden-migh...
4,"Calif., N.Y. and Texas win big in new Biden cl...",E&E News,2 weeks ago,https://www.eenews.net/articles/calif-n-y-and-...


In [9]:
harvey_df.to_csv('nlp_data/harvey_links.csv', index=False)

In [8]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

In [9]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\jared\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [10]:
urls = harvey_df['Link']
articles = []
for url in urls:
    browser.visit(url)
    html = browser.html
    soup = bs(html, "html.parser")
    whitelist = ['p']
    text_elements = [t for t in soup.find_all(text=True) if t.parent.name in whitelist]
    articles.append(text_elements)

In [11]:
harvey_df['text'] = articles
harvey_df.to_csv('nlp_data/harvey_articles.csv', index=False)

In [12]:
harvey_df

,Title,Source,Date,Link,text
0,FEMA overhauls the National Flood Insurance Pr...,CNBC,2 days ago,https://www.cnbc.com/2021/08/17/fema-overhauls...,[Climate change and its devastating impact are...
1,Fix Disaster Response Now,Scientific American,2 days ago,https://www.scientificamerican.com/article/fix...,[The True Haiti Earthquake Death Toll Is Much ...
2,"Meet Marcus Coleman, New Director of DHS Cente...",| FEMA.gov,2 weeks ago,https://www.fema.gov/blog/meet-marcus-coleman-...,[An official website of the United States gove...
3,Why Biden might be spending too much on climate,E&E News,1 day ago,https://www.eenews.net/articles/why-biden-migh...,"[By , | 08/18/2021 06:25 AM EST , A park ben..."
4,"Calif., N.Y. and Texas win big in new Biden cl...",E&E News,2 weeks ago,https://www.eenews.net/articles/calif-n-y-and-...,"[By , | 08/06/2021 05:45 AM EST , Debris fro..."
...,...,...,...,...,...
95,FEMA: Recovery resource fair planned for Calca...,KPLC,"Apr 8, 2021",https://www.kplctv.com/2021/04/07/fema-recover...,"[LAKE CHARLES, La. (KPLC) - The Southwest Long..."
96,Supplemental Material for GAO-20-503: Select D...,Government Accountability Office,"Sep 30, 2020",https://www.gao.gov/products/gao-20-674sp,"[After a disaster, FEMA's Individuals and Hous..."
97,2019-2020 State Legislative Report on Natural ...,National Conference of State Legislatures,"Jan 19, 2021",https://www.ncsl.org/default.aspx?tabid=35851,[As natural disasters increase in frequency an...
98,Hurricane Harvey is one of 74 disasters declar...,Orange County Register,"Aug 29, 2017",https://www.ocregister.com/2017/08/29/hurrican...,"[ , , \n\t\t\tTrending:\t\t, The , led the F..."


In [14]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
sentiment = lambda article: vader.polarity_scores(article)['compound']
#harvey_df['sentiment_score'] = .apply(sentiment).round(3)

In [26]:
compound_sentiment = []
for article in articles:
    line_scores = []
    for line in article:
        compound = vader.polarity_scores(line)['compound']
        line_scores.append(compound)
    compound_sum = sum(line_scores) / len(line_scores)
    compound_sentiment.append(compound_sum)

In [27]:
harvey_df['sentiment'] = compound_sentiment
harvey_df.head(10)

,Title,Source,Date,Link,text,sentiment
0,FEMA overhauls the National Flood Insurance Pr...,CNBC,2 days ago,https://www.cnbc.com/2021/08/17/fema-overhauls...,[Climate change and its devastating impact are...,0.016454
1,Fix Disaster Response Now,Scientific American,2 days ago,https://www.scientificamerican.com/article/fix...,[The True Haiti Earthquake Death Toll Is Much ...,-0.060244
2,"Meet Marcus Coleman, New Director of DHS Cente...",| FEMA.gov,2 weeks ago,https://www.fema.gov/blog/meet-marcus-coleman-...,[An official website of the United States gove...,0.213000
3,Why Biden might be spending too much on climate,E&E News,1 day ago,https://www.eenews.net/articles/why-biden-migh...,"[By , | 08/18/2021 06:25 AM EST , A park ben...",-0.017787
4,"Calif., N.Y. and Texas win big in new Biden cl...",E&E News,2 weeks ago,https://www.eenews.net/articles/calif-n-y-and-...,"[By , | 08/06/2021 05:45 AM EST , Debris fro...",-0.099705
5,Lots of homeowners are dropping flood insuranc...,Marketplace.org,2 weeks ago,https://www.marketplace.org/2021/08/04/homeown...,"[\n, \n, \n, More than a quarter of the homeow...",-0.131635
6,Texas Coastal Residents On Alert As Atlantic H...,Houston Public Media,1 week ago,https://www.houstonpublicmedia.org/articles/ne...,"[There are only a few days left of our , . You...",0.153658
7,Jefferson County Man Guilty of FEMA Fraud Rela...,Department of Justice,1 month ago,https://www.justice.gov/usao-edtx/pr/jefferson...,"[BEAUMONT, Texas – A Port Arthur man has plead...",-0.163145
8,Hurricane Harvey Housing Assistance Lacked Nec...,Homeland Security Today,1 month ago,https://www.hstoday.us/federal-pages/governmen...,[Due to the severity of multiple disasters in ...,0.226014
9,FEMA Assistance Is Unfair To Poorer Disaster S...,NPR,1 month ago,https://www.npr.org/2021/06/29/1004347023/why-...,"[\n, \n, \n Many survivors of c...",-0.218790


In [29]:
harvey_df.to_csv("nlp_data/harvey_sentiment.csv")

In [28]:
harvey_sentiment = sum(harvey_df['sentiment']) / len(harvey_df['sentiment'])
harvey_sentiment

-0.010162541511090363

In [4]:
harvey_df = pd.read_csv('nlp_data/harvey_articles.csv')
article0_text = harvey_df['text'][15].replace("['", "").replace("]'","").split("',")
article0_text[1]

" ' delivered the annual convocation address, Aug. 17, in the Montagne Center. The convocation was held simultaneously online and in person for the second year in a row. This year’s theme incorporated the “LU Stronger Together” campaign.\\xa0"